In [1]:
import sys
import os
import os.path
from pathlib import Path
from rdflib import Graph, Namespace, RDF, URIRef, Literal
from collections import Counter
import json

import rdflib

import pandas as pd
import numpy as np

import laspy

from scipy.spatial.distance import euclidean

import geomapi.tools as tl
from geomapi.nodes import PointCloudNode
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.nodes import PointCloudNode

from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA

import topologicpy as tp
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary

from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex

from topologicpy.Vector import Vector
from topologicpy.Plotly import Plotly

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import context_KUL
import utils_KUL as kul
import utils_energy as en

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


STANDARD INPUTS

In [24]:
current_directory=Path(os.getcwd())
print(current_directory)

name = 'school_v1'

path = Path(os.getcwd()).parents[0] / 'data'
pcd_input_path = path / f'{name}'
file_name = pcd_input_path / f'{name}.laz'
print(f'Pcd_input_path: {pcd_input_path}\nFile name {file_name}')

class_file = Path(os.getcwd()) / '_classes_expanded.json'
print('class_file:', class_file)

if not class_file.exists():
    print(f"class file not found: {class_file}")
else:
    print(f"class file exists: {class_file}")


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script
Pcd_input_path: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\school_v1
File name c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\school_v1\school_v1.laz
class_file: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\_classes_expanded.json
class file exists: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\script\_classes_expanded.json


In [6]:
output_folder = Path(os.getcwd()).parents[1] / 'outputs'
json_output_path = os.path.join(output_folder, f'{name}_elemnts.json') 
geometry_output_path = os.path.join(output_folder, f'{name}_elements.obj') # these are the bounding surfaces of the reference levels (optional)

print(pcd_input_path)


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\school_v1


In [7]:
# Output 
file_graph_name = name
graph_path_str = str(output_folder / f'{file_graph_name}_graph.ttl')
print('Output folder', output_folder)


Output folder c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\outputs


In [9]:
from rdflib import Graph

input_folder_ttl = path / 'graph_files'
print('Input folder ttl:', input_folder_ttl)

graph_path = input_folder_ttl / f'{name}_graph.ttl'
print('ttl:', graph_path)
graph_path_uri = graph_path.as_uri()
print('ttl URI:', graph_path_uri)

graph = Graph()
graph.parse(graph_path_uri, format="turtle")
print("Graph parsed successfully")


Input folder ttl: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\graph_files
ttl: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\graph_files\school_v1_graph.ttl
ttl URI: file:///c:/Users/oscar/OneDrive%20-%20Fondazione%20Bruno%20Kessler/KUL_GeometricModel/data/graph_files/school_v1_graph.ttl
Graph parsed successfully


In [10]:
output_folder = path / 'outputs' 
print(output_folder)
output_file_path = output_folder / f'{name}.json'
output_json_file = output_folder = path / f'{name}_elements.json'


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\KUL_GeometricModel\data\outputs


In [11]:
with open(class_file, 'r') as file:
    json_data = json.load(file)

class_dict = {
    'classes': json_data.get('classes', []),
    'default': json_data.get('default', {}),
    'type': json_data.get('type', ''),
    'format': json_data.get('format', ''),
    'created_with': json_data.get('created_with', '')
}

print('Class dictionary:', class_dict)

Class dictionary: {'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}, {'name': 'Lights', 'id': 6, 'temp_id': 6, 'color': '#7a93eb'}, {'name': 'Radiators', 'id': 7, 'temp_id': 7, 'color': '#d93b67'}, {'name': 'HVAC', 'id': 8, 'temp_id': 8, 'color': '#42b8a6'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


In [12]:
laz, pcd, pcd_nodes, normals = kul.load_point_cloud(file_name)
print(f'Las file {laz}\nPCD {pcd}\n PCD_Nodes {pcd_nodes}\n Normals {normals}')

Las file <LasData(1.2, point fmt: <PointFormat(2, 2 bytes of extra dims)>, 3126542 points, 1 vlrs)>
PCD PointCloud with 3126542 points.
 PCD_Nodes <geomapi.nodes.pointcloudnode.PointCloudNode object at 0x00000227D00CB910>
 Normals [[-0.04697317 -0.03550417  0.99826498]
 [-0.01021667  0.02556956  0.99962084]
 [ 0.03704321 -0.02480083  0.99900587]
 ...
 [-0.9998618  -0.01634594  0.0030307 ]
 [ 0.99996751 -0.0048734   0.00642091]
 [ 0.99892968  0.00289677  0.04616383]]


ENERGY INPUTS: IMPORT IDF

In [46]:
idf_data = """
Building,
  MyBuilding,             !- Name
  30.0,                   !- North Axis (degrees)
  Suburbs,                !- Terrain
  0.04,                   !- Loads Convergence Tolerance Value
  0.004,                  !- Temperature Convergence Tolerance Value (deltaC)
  FullExterior,           !- Solar Distribution
  25;                     !- Maximum Number of Warmup Days

Zone,
  MyZone,                 !- Name
  0.0000,                 !- Direction of Relative North (degrees)
  0.0000,                 !- X Origin (m)
  0.0000,                 !- Y Origin (m)
  0.0000,                 !- Z Origin (m)
  1,                      !- Type
  1,                      !- Multiplier
  autocalculate,          !- Ceiling Height (m)
  autocalculate;          !- Volume (m3)

Material,
  Concrete,               !- Name
  Rough,                  !- Roughness
  0.1524,                 !- Thickness (m)
  1.311,                  !- Conductivity (W/m-K)
  2240,                   !- Density (kg/m3)
  836.8;                  !- Specific Heat (J/kg-K)

WindowMaterial:SimpleGlazingSystem,
  WindowGlass,            !- Name
  2.5,                    !- U-Factor (W/m2-K)
  0.6,                    !- Solar Heat Gain Coefficient
  0.7;                    !- Visible Transmittance

Schedule:Compact,
  LightingSchedule,       !- Name
  Fraction,               !- Schedule Type Limits Name
  Through: 12/31,         !- Field 1
  For: AllDays,           !- Field 2
  Until: 24:00, 1.0;      !- Field 3

Lights,
  MyZone_Lights,          !- Name
  MyZone,                 !- Zone Name
  LightingSchedule,       !- Schedule Name
  Watts/Area,             !- Design Level Calculation Method
  ,                       !- Lighting Level (W)
  12.0,                   !- Watts per Zone Floor Area (W/m2)
  ,                       !- Watts per Person
  0.2000,                 !- Return Air Fraction
  0.0000,                 !- Fraction Radiant
  0.0000,                 !- Fraction Visible
  1.0000,                 !- Fraction Replaceable
  GeneralLights;          !- End-Use Subcategory
"""


BUILDING METADATA

Substitute laz with ComplexCells and Clusters from TopologicEnergyModel.ipynb

In [36]:
def compute_oriented_bounding_box(laz_file_path):

    # Open and read the LAZ file
    las_data = laspy.read(laz_file_path)
    
    # Extract the X, Y, and Z coordinates (these are already scaled)
    x_coords = las_data.x
    y_coords = las_data.y
    z_coords = las_data.z
    
    # Stack the coordinates into a (N, 3) array of points
    points = np.vstack((x_coords, y_coords, z_coords)).T
    
    # Perform PCA to find the principal axes of the point cloud
    pca = PCA(n_components=3)
    pca.fit(points)
    
    # The principal axes (eigenvectors)
    rotation_matrix = pca.components_
    
    # Transform points into the principal component space (rotated points)
    points_rotated = pca.transform(points)
    
    # Calculate the min and max in the rotated space (principal component space)
    min_corner = np.min(points_rotated, axis=0)
    max_corner = np.max(points_rotated, axis=0)
    
    # Create the bounding box in the rotated space
    bbox_rotated = np.array([min_corner[0], max_corner[0],  # X axis
                             min_corner[1], max_corner[1],  # Y axis
                             min_corner[2], max_corner[2]]) # Z axis
    
    return points, points_rotated, bbox_rotated, rotation_matrix

def extract_beta_angle_from_rotation_matrix(rotation_matrix):

    # Extract the β angle (rotation around the Z-axis)
    beta_rad = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
    
    # Convert the angle from radians to degrees
    beta_angle = np.degrees(beta_rad)
    
    return beta_angle

def rotation_matrix_z(beta_angle):

    # Convert the angle to radians
    beta_rad = np.radians(beta_angle)
    
    # 3D rotation matrix around Z-axis
    rotation_matrix = np.array([
        [np.cos(beta_rad), -np.sin(beta_rad), 0],
        [np.sin(beta_rad), np.cos(beta_rad),  0],
        [0, 0, 1]
    ])
    
    return rotation_matrix

In [37]:
points, points_rotated, bbox_rotated, rotation_matrix = compute_oriented_bounding_box(file_name)
rotation_matrix_inverse = np.linalg.inv(rotation_matrix)
center = np.mean(points, axis = 0)

points_aligned_with_axes = kul.rotate_points_3d(points_rotated, center, rotation_matrix_inverse)

# Display results
print("Rotated Points (first 5):", points_rotated[:5])  # Displaying first 5 points for brevity
print("\nBounding Box (Rotated Space):", bbox_rotated)
print("\nRotation Matrix:")
print(rotation_matrix)


Rotated Points (first 5): [[11.59487928 -3.62211859 -1.2508574 ]
 [11.63669688 -3.59472121 -1.25249207]
 [11.50901361 -3.63361402 -1.25614333]
 [11.64640303 -3.55973628 -1.24919881]
 [11.61046311 -3.56596577 -1.2507849 ]]

Bounding Box (Rotated Space): [-11.07647717  11.81420093  -4.17320686   4.34909341  -1.56328407
   2.54370522]

Rotation Matrix:
[[-0.99991585  0.00777841 -0.01038176]
 [-0.00742107 -0.99939339 -0.03402616]
 [-0.01064013 -0.03394625  0.99936702]]


Building's orientation or the surface azimuth angle (β)

In [43]:
beta_angle = extract_beta_angle_from_rotation_matrix(rotation_matrix)

rotation_matrix_building = rotation_matrix_z(beta_angle)
center_of_bbox = np.mean(points, axis=0)  # Assuming you rotate around the center of the building
rotated_bbox = kul.rotate_points_3d(points_rotated, center_of_bbox, rotation_matrix_building)

print (f"Exposition angle of buildings N/S: {round(beta_angle, 3)}")
print(f"Building's Rotated Bounding Box (after applying β angle rotation): {rotated_bbox[:5]}") 
print(f'Rotation Matrix used for β angle rotation: {rotation_matrix_building}')


Exposition angle of buildings N/S: -179.575
Building's Rotated Bounding Box (after applying β angle rotation): [[-95.82635734  25.02728095  -1.2508574 ]
 [-95.86837711  25.00019467  -1.25249207]
 [-95.74040872  25.03813882  -1.25614333]
 [-95.87834264  24.96528274  -1.24919881]
 [-95.84235748  24.97124534  -1.2507849 ]]
Rotation Matrix used for β angle rotation: [[-0.99997246  0.00742149  0.        ]
 [-0.00742149 -0.99997246  0.        ]
 [ 0.          0.          1.        ]]
